In [1]:
import pandas as pd
import scipy.stats as stats

from sklearn import metrics, model_selection
from catboost import CatBoostClassifier

In [2]:
df_train = pd.read_csv('../../Feature_Encoding/data/train_js_encoding.csv')
df_test = pd.read_csv('../../Feature_Encoding/data/test_js_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [4]:
#Realiza busqueda completa combinando los parametros
def catboost(x_train, y_train, x_validation, y_validation):
    catb_classifier = CatBoostClassifier()
    params_catb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.05, 0.1], 'l2_leaf_reg': [0, 1, 5]}    
    catb_gs = model_selection.GridSearchCV(catb_classifier, params_catb, cv=5)
    catb_gs.fit(x_train, y_train)
    catb_best = catb_gs.best_estimator_
    print(catb_gs.best_params_)
    print('catb: {}'.format(catb_best.score(x_validation, y_validation)))
    return catb_best

In [5]:
#Realiza busqueda random dentro de los parametros validos
def catboost2(x_train, y_train, x_validation, y_validation):
    catb_classifier = CatBoostClassifier()
    params_catb_2={
        'n_estimators':stats.randint(10,500),'learning_rate':stats.uniform(0.01,0.3),
        'subsample':stats.uniform(0.3,0.7),'l2_leaf_reg':[1,5,10],
        'max_depth':[3,10,6],'colsample_bylevel':stats.uniform(0.,0.6)
    }
    
    catb_rs = model_selection.RandomizedSearchCV(CatBoostClassifier(),
                          param_distributions=params_catb_2,
                          cv=2,
                          scoring='neg_log_loss',
                          verbose=1,
                          n_iter=200)
    
    catb_rs.fit(x_train, y_train)
    catb_best = catb_rs.best_estimator_
    print(catb_rs.best_params_)
    print('catb: {}'.format(catb_best.score(x_validation, y_validation)))
    return catb_best

In [6]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = metrics.log_loss(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [7]:
y = train.Target
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(df_train, y, test_size=0.3, stratify=y)

In [ ]:
catboost_2 = catboost2(x_train, y_train, x_validation, y_validation)
test_model(catboost_2, x_validation, y_validation)
cross_val(catboost_2, x_train, y_train)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
0:	learn: 0.6657701	total: 89.1ms	remaining: 29.2s
1:	learn: 0.6411453	total: 133ms	remaining: 21.8s
2:	learn: 0.6165455	total: 169ms	remaining: 18.4s
3:	learn: 0.5954149	total: 204ms	remaining: 16.6s
4:	learn: 0.5743355	total: 243ms	remaining: 15.7s
5:	learn: 0.5546757	total: 279ms	remaining: 15s
6:	learn: 0.5358379	total: 310ms	remaining: 14.2s
7:	learn: 0.5197313	total: 351ms	remaining: 14.1s
8:	learn: 0.5031780	total: 374ms	remaining: 13.3s
9:	learn: 0.4893317	total: 409ms	remaining: 13.1s
10:	learn: 0.4767283	total: 446ms	remaining: 12.9s
11:	learn: 0.4643644	total: 477ms	remaining: 12.6s
12:	learn: 0.4516813	total: 503ms	remaining: 12.2s
13:	learn: 0.4429627	total: 512ms	remaining: 11.5s
14:	learn: 0.4333743	total: 544ms	remaining: 11.4s
15:	learn: 0.4219289	total: 571ms	remaining: 11.2s
16:	learn: 0.4135761	total: 599ms	remaining: 11s
17:	learn: 0.4046100	total: 632ms	remaining: 10.9s
18:	learn: 0.3970866	total: 672m

160:	learn: 0.1587460	total: 5.5s	remaining: 5.74s
161:	learn: 0.1584103	total: 5.54s	remaining: 5.71s
162:	learn: 0.1578522	total: 5.57s	remaining: 5.67s
163:	learn: 0.1568983	total: 5.6s	remaining: 5.63s
164:	learn: 0.1560245	total: 5.63s	remaining: 5.6s
165:	learn: 0.1553293	total: 5.66s	remaining: 5.56s
166:	learn: 0.1548518	total: 5.69s	remaining: 5.52s
167:	learn: 0.1545500	total: 5.73s	remaining: 5.49s
168:	learn: 0.1539160	total: 5.76s	remaining: 5.46s
169:	learn: 0.1535221	total: 5.79s	remaining: 5.42s
170:	learn: 0.1532565	total: 5.82s	remaining: 5.38s
171:	learn: 0.1525022	total: 5.85s	remaining: 5.34s
172:	learn: 0.1517537	total: 5.89s	remaining: 5.31s
173:	learn: 0.1513998	total: 5.92s	remaining: 5.27s
174:	learn: 0.1509785	total: 5.96s	remaining: 5.25s
175:	learn: 0.1505108	total: 6.01s	remaining: 5.22s
176:	learn: 0.1502056	total: 6.03s	remaining: 5.18s
177:	learn: 0.1498266	total: 6.07s	remaining: 5.15s
178:	learn: 0.1490217	total: 6.11s	remaining: 5.12s
179:	learn: 0.1

323:	learn: 0.0973025	total: 11.4s	remaining: 175ms
324:	learn: 0.0971534	total: 11.4s	remaining: 140ms
325:	learn: 0.0969677	total: 11.4s	remaining: 105ms
326:	learn: 0.0965707	total: 11.5s	remaining: 70.2ms
327:	learn: 0.0963767	total: 11.5s	remaining: 35.1ms
328:	learn: 0.0960029	total: 11.5s	remaining: 0us
0:	learn: 0.6698795	total: 55.6ms	remaining: 18.3s
1:	learn: 0.6444024	total: 94.7ms	remaining: 15.5s
2:	learn: 0.6229285	total: 125ms	remaining: 13.6s
3:	learn: 0.6023208	total: 173ms	remaining: 14.1s
4:	learn: 0.5811691	total: 221ms	remaining: 14.3s
5:	learn: 0.5633592	total: 264ms	remaining: 14.2s
6:	learn: 0.5461652	total: 297ms	remaining: 13.7s
7:	learn: 0.5315686	total: 336ms	remaining: 13.5s
8:	learn: 0.5146558	total: 367ms	remaining: 13.1s
9:	learn: 0.4993267	total: 414ms	remaining: 13.2s
10:	learn: 0.4863175	total: 451ms	remaining: 13s
11:	learn: 0.4734524	total: 497ms	remaining: 13.1s
12:	learn: 0.4596944	total: 535ms	remaining: 13s
13:	learn: 0.4513200	total: 543ms	rem

155:	learn: 0.1688045	total: 6.64s	remaining: 7.36s
156:	learn: 0.1683768	total: 6.67s	remaining: 7.3s
157:	learn: 0.1675802	total: 6.71s	remaining: 7.26s
158:	learn: 0.1672631	total: 6.75s	remaining: 7.22s
159:	learn: 0.1668006	total: 6.79s	remaining: 7.17s
160:	learn: 0.1661584	total: 6.85s	remaining: 7.15s
161:	learn: 0.1658272	total: 6.91s	remaining: 7.12s
162:	learn: 0.1646516	total: 6.95s	remaining: 7.08s
163:	learn: 0.1642613	total: 6.97s	remaining: 7.02s
164:	learn: 0.1636096	total: 7s	remaining: 6.96s
165:	learn: 0.1628832	total: 7.06s	remaining: 6.93s
166:	learn: 0.1624803	total: 7.09s	remaining: 6.88s
167:	learn: 0.1613581	total: 7.13s	remaining: 6.83s
168:	learn: 0.1610798	total: 7.17s	remaining: 6.78s
169:	learn: 0.1599794	total: 7.21s	remaining: 6.74s
170:	learn: 0.1596617	total: 7.24s	remaining: 6.69s
171:	learn: 0.1591005	total: 7.28s	remaining: 6.65s
172:	learn: 0.1587702	total: 7.31s	remaining: 6.59s
173:	learn: 0.1584618	total: 7.35s	remaining: 6.54s
174:	learn: 0.15

316:	learn: 0.1047234	total: 12.8s	remaining: 485ms
317:	learn: 0.1045453	total: 12.8s	remaining: 444ms
318:	learn: 0.1043047	total: 12.9s	remaining: 404ms
319:	learn: 0.1041450	total: 12.9s	remaining: 364ms
320:	learn: 0.1038597	total: 13s	remaining: 324ms
321:	learn: 0.1036955	total: 13s	remaining: 283ms
322:	learn: 0.1034636	total: 13s	remaining: 242ms
323:	learn: 0.1033116	total: 13.1s	remaining: 202ms
324:	learn: 0.1031316	total: 13.1s	remaining: 162ms
325:	learn: 0.1029444	total: 13.2s	remaining: 121ms
326:	learn: 0.1026769	total: 13.2s	remaining: 80.8ms
327:	learn: 0.1024485	total: 13.3s	remaining: 40.4ms
328:	learn: 0.1023295	total: 13.3s	remaining: 0us
0:	learn: 0.6570781	total: 3.95ms	remaining: 774ms
1:	learn: 0.6417862	total: 10.9ms	remaining: 1.06s
2:	learn: 0.5773136	total: 14.1ms	remaining: 913ms
3:	learn: 0.5768199	total: 16.5ms	remaining: 797ms
4:	learn: 0.5281357	total: 19.7ms	remaining: 756ms
5:	learn: 0.5048116	total: 27.7ms	remaining: 882ms
6:	learn: 0.4837697	tota

...

349:	learn: 0.0034464	total: 7.2s	remaining: 2.32s
350:	learn: 0.0034463	total: 7.21s	remaining: 2.3s
351:	learn: 0.0034463	total: 7.23s	remaining: 2.28s
352:	learn: 0.0034463	total: 7.24s	remaining: 2.26s
353:	learn: 0.0034462	total: 7.26s	remaining: 2.23s
354:	learn: 0.0034462	total: 7.28s	remaining: 2.21s
355:	learn: 0.0034462	total: 7.3s	remaining: 2.19s
356:	learn: 0.0034462	total: 7.32s	remaining: 2.17s
357:	learn: 0.0034462	total: 7.33s	remaining: 2.15s
358:	learn: 0.0034462	total: 7.34s	remaining: 2.13s
359:	learn: 0.0034462	total: 7.36s	remaining: 2.1s
360:	learn: 0.0034461	total: 7.38s	remaining: 2.08s
361:	learn: 0.0034079	total: 7.39s	remaining: 2.06s
362:	learn: 0.0034079	total: 7.4s	remaining: 2.04s
363:	learn: 0.0034078	total: 7.42s	remaining: 2.02s
364:	learn: 0.0034078	total: 7.43s	remaining: 2s
365:	learn: 0.0034078	total: 7.45s	remaining: 1.97s
366:	learn: 0.0034078	total: 7.46s	remaining: 1.95s
367:	learn: 0.0034078	total: 7.49s	remaining: 1.93s
368:	learn: 0.003407

45:	learn: 0.0336553	total: 831ms	remaining: 7.54s
46:	learn: 0.0330880	total: 877ms	remaining: 7.76s
47:	learn: 0.0322281	total: 906ms	remaining: 7.84s
48:	learn: 0.0313143	total: 921ms	remaining: 7.78s
49:	learn: 0.0297288	total: 958ms	remaining: 7.91s
50:	learn: 0.0290143	total: 987ms	remaining: 7.97s
51:	learn: 0.0285113	total: 1.01s	remaining: 8.02s
52:	learn: 0.0273939	total: 1.03s	remaining: 8s
53:	learn: 0.0266504	total: 1.09s	remaining: 8.26s
54:	learn: 0.0259150	total: 1.12s	remaining: 8.34s
55:	learn: 0.0249371	total: 1.15s	remaining: 8.35s
56:	learn: 0.0241083	total: 1.18s	remaining: 8.39s
57:	learn: 0.0235577	total: 1.2s	remaining: 8.39s
58:	learn: 0.0227359	total: 1.23s	remaining: 8.42s
59:	learn: 0.0223656	total: 1.25s	remaining: 8.41s
60:	learn: 0.0219023	total: 1.28s	remaining: 8.45s
61:	learn: 0.0212144	total: 1.31s	remaining: 8.49s
62:	learn: 0.0205408	total: 1.34s	remaining: 8.49s
63:	learn: 0.0202370	total: 1.37s	remaining: 8.52s
64:	learn: 0.0196505	total: 1.4s	re

211:	learn: 0.0035525	total: 4.99s	remaining: 5.9s
212:	learn: 0.0035032	total: 5s	remaining: 5.87s
213:	learn: 0.0035032	total: 5.01s	remaining: 5.83s
214:	learn: 0.0035032	total: 5.04s	remaining: 5.81s
215:	learn: 0.0035031	total: 5.06s	remaining: 5.79s
216:	learn: 0.0035031	total: 5.08s	remaining: 5.76s
217:	learn: 0.0034574	total: 5.1s	remaining: 5.73s
218:	learn: 0.0034574	total: 5.12s	remaining: 5.7s
219:	learn: 0.0034574	total: 5.13s	remaining: 5.67s
220:	learn: 0.0034574	total: 5.15s	remaining: 5.64s
221:	learn: 0.0034164	total: 5.17s	remaining: 5.61s
222:	learn: 0.0034163	total: 5.18s	remaining: 5.58s
223:	learn: 0.0034163	total: 5.2s	remaining: 5.55s
224:	learn: 0.0034163	total: 5.22s	remaining: 5.53s
225:	learn: 0.0034163	total: 5.24s	remaining: 5.49s
226:	learn: 0.0034161	total: 5.25s	remaining: 5.46s
227:	learn: 0.0034161	total: 5.28s	remaining: 5.44s
228:	learn: 0.0033805	total: 5.3s	remaining: 5.41s
229:	learn: 0.0033805	total: 5.32s	remaining: 5.39s
230:	learn: 0.003380

372:	learn: 0.0031352	total: 8.85s	remaining: 2.13s
373:	learn: 0.0031351	total: 8.86s	remaining: 2.11s
374:	learn: 0.0031351	total: 8.88s	remaining: 2.08s
375:	learn: 0.0031346	total: 8.89s	remaining: 2.06s
376:	learn: 0.0030754	total: 8.91s	remaining: 2.03s
377:	learn: 0.0030752	total: 8.93s	remaining: 2.01s
378:	learn: 0.0030399	total: 8.94s	remaining: 1.98s
379:	learn: 0.0030399	total: 8.96s	remaining: 1.96s
380:	learn: 0.0030399	total: 8.98s	remaining: 1.93s
381:	learn: 0.0030399	total: 8.99s	remaining: 1.91s
382:	learn: 0.0030399	total: 9.01s	remaining: 1.88s
383:	learn: 0.0030398	total: 9.03s	remaining: 1.86s
384:	learn: 0.0030398	total: 9.05s	remaining: 1.83s
385:	learn: 0.0030398	total: 9.08s	remaining: 1.81s
386:	learn: 0.0030398	total: 9.1s	remaining: 1.79s
387:	learn: 0.0030398	total: 9.12s	remaining: 1.76s
388:	learn: 0.0030398	total: 9.14s	remaining: 1.74s
389:	learn: 0.0030397	total: 9.15s	remaining: 1.71s
390:	learn: 0.0030397	total: 9.17s	remaining: 1.69s
391:	learn: 0

69:	learn: 0.0369928	total: 1.46s	remaining: 2.17s
70:	learn: 0.0361744	total: 1.48s	remaining: 2.15s
71:	learn: 0.0346562	total: 1.51s	remaining: 2.15s
72:	learn: 0.0340303	total: 1.54s	remaining: 2.13s
73:	learn: 0.0333874	total: 1.56s	remaining: 2.11s
74:	learn: 0.0326868	total: 1.59s	remaining: 2.1s
75:	learn: 0.0321939	total: 1.61s	remaining: 2.08s
76:	learn: 0.0314441	total: 1.64s	remaining: 2.06s
77:	learn: 0.0307140	total: 1.66s	remaining: 2.04s
78:	learn: 0.0302588	total: 1.68s	remaining: 2.03s
79:	learn: 0.0296106	total: 1.71s	remaining: 2.01s
80:	learn: 0.0289336	total: 1.74s	remaining: 1.99s
81:	learn: 0.0284170	total: 1.76s	remaining: 1.97s
82:	learn: 0.0277848	total: 1.78s	remaining: 1.96s
83:	learn: 0.0274955	total: 1.82s	remaining: 1.95s
84:	learn: 0.0271207	total: 1.84s	remaining: 1.93s
85:	learn: 0.0262712	total: 1.86s	remaining: 1.9s
86:	learn: 0.0257171	total: 1.88s	remaining: 1.88s
87:	learn: 0.0252644	total: 1.92s	remaining: 1.87s
88:	learn: 0.0249429	total: 1.94s

In [ ]:
y_pred = catboost_2.predict_proba(df_test)[:,1]
submission_catboost = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred})
submission_catboost = submission_catboost.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_catboost.to_csv('../submits/js_catboost.csv', index=False)